In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from pickle import dump, load



device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'

In [2]:
print(f'running on {device}')
file_location = os.getcwd()

running on cuda


In [3]:
#Load in data scaling
scaler = load(open(file_location + '/scaler_cuda.pkl','rb'))

#Load in label encoding
le  = LabelEncoder()
le.classes_ = np.load(file_location + '/encoding_cuda.npy', allow_pickle=True)



In [4]:
class testData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [6]:
#load in model, model must be loaded in after its class is instantiated
#otherwise you will get a serialization error
model = torch.load(file_location + '/avacado_model_cuda.pt')

In [7]:
#read in files
file_in = os.getcwd() + "/avacados/avac_sample.csv"
df = pd.read_csv(file_in)

df = df.drop(columns=['Date', 'XLarge Bags'])

print(df.head())


   Unnamed: 0  AveragePrice  Total Volume        4046       4225      4770  \
0           0          1.07     417232.18   278048.26   62485.97    714.93   
1           1          1.10     454702.00   382900.99   19543.18    522.81   
2           2          2.03       1794.39     1069.54     187.76      0.00   
3           3          1.38    1975524.70   833904.89  499191.31  10560.99   
4           4          1.16    2197763.70  1420318.78  298081.99  25682.97   

   Total Bags  Small Bags  Large Bags          type  year       region  
0    75983.02    46290.32    29678.76  conventional  2015      Atlanta  
1    51735.02    40505.16    11199.95  conventional  2015      Atlanta  
2      537.09      500.00       37.09       organic  2016  GrandRapids  
3   631867.51   584294.01    29543.77  conventional  2017       Plains  
4   453679.96   309652.75   143978.69  conventional  2015    Southeast  


In [8]:
#Load data
#print(df.head)
X = df.iloc[:, df.columns != 'AveragePrice'] #grab first 13 elements for input
y = df.loc[:, df.columns == 'AveragePrice']   #seperate last element (target values)

le = LabelEncoder()
#scaler = StandardScaler()
X['type'] = le.fit_transform(X['type'].astype(str))
X['region'] = le.fit_transform(X['region'].astype(str))
X_test = scaler.fit_transform(X)

print(y)

   AveragePrice
0          1.07
1          1.10
2          2.03
3          1.38
4          1.16
5          1.13
6          1.13
7          1.17
8          1.02
9          2.39


<ipython-input-8-b81455a16b04>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['type'] = le.fit_transform(X['type'].astype(str))
<ipython-input-8-b81455a16b04>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['region'] = le.fit_transform(X['region'].astype(str))


In [9]:
#reshape data into a tensor of floats
test_data = testData(torch.FloatTensor(X_test),torch.FloatTensor(y.values))
dataloader = DataLoader(dataset=test_data, batch_size=1)



In [10]:
size = len(dataloader.dataset)
num_batches = len(dataloader)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        print(f'''The predicted model generated {round(pred.item(),5)}\t actual was {round(y.item(), 5)} \t difference was {round((pred-y).item(),5)}''')

The predicted model generated 27.95205	 actual was 1.07 	 difference was 26.88205
The predicted model generated 28.88719	 actual was 1.1 	 difference was 27.78719
The predicted model generated 44.60019	 actual was 2.03 	 difference was 42.57019
The predicted model generated 1.22165	 actual was 1.38 	 difference was -0.15835
The predicted model generated 1.14346	 actual was 1.16 	 difference was -0.01654
The predicted model generated 1.31009	 actual was 1.13 	 difference was 0.18009
The predicted model generated 27.73861	 actual was 1.13 	 difference was 26.60861
The predicted model generated 11.79626	 actual was 1.17 	 difference was 10.62626
The predicted model generated 0.92449	 actual was 1.02 	 difference was -0.09551
The predicted model generated 36.7055	 actual was 2.39 	 difference was 34.3155
